In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
def getBdate(bdate):
  return datetime.strptime(bdate, '%d.%m.%Y').month

In [3]:
# количество признаков входного слоя
feature_count = 5

drop_column = ['№п/п', 'Основание поступления']
gender_dict = {'Мужской': 1,'Женский': 0}

# подготовленный датасет
data = []

df = pd.read_csv('data/input.csv', sep=',')
df.drop(drop_column, axis = 1, inplace = True)

# переводим пол в числовой формат, т. к. это будет один из признаков
df['Пол'].replace(gender_dict, inplace = True) 

# приводим баллы по ЕГЭ к диапозону от 0 до 1
df.astype({'Баллы ЕГЭ': 'float'}).dtypes

fios = df['ФИО'].unique()
subjects = df['Предметы ЕГЭ'].unique()
cities = df['Населенный пункт по прописке'].unique()
schools = df['Учебное заведение'].unique()
classes = df['Специальность/направление'].unique()

for el in fios:
    student = df[df['ФИО'] == el]
    features = np.zeros((feature_count))
    for i in range(len(subjects)):
        res = student.loc[student['Предметы ЕГЭ'] == subjects[i], 'Баллы ЕГЭ'].values
        features[i] = res[0] if len(res) else 0
#     features[5] = student.iloc[0]['Пол']
#     features[6] = getBdate(student.iloc[0]['Дата рождения'])
    
#     city_index = np.where(cities == student.iloc[0]['Населенный пункт по прописке'])[0]
#     features[7] = 0 if len(city_index) == 0 else city_index[0]
    
#     school_number = np.where(schools == student.iloc[0]['Учебное заведение'])[0]
#     features[8] = 0 if len(school_number) == 0 else school_number[0]
    
#     for i in range(len(features)):
#         sum = 0 
#         for j in range(len(features)):
#             sum += features[j] ** 2
#         features[i] = features[i] / math.sqrt(sum)
#     features = (features * features.mean()) / features.std()
    res = np.zeros((len(classes)))
    res[np.where(classes == student.iloc[0]['Специальность/направление'])[0].item(0)] = 1
    
    temp_data = dict(fio = el, features = features, result = res)
    
    data.append(temp_data)

# Результаты ЕГЭ

## Русский, математика, информатика

In [4]:
computer_science_classes = df.loc[df['Предметы ЕГЭ'] == 'Информатика и ИКТ', 'Специальность/направление'].unique()
nature_science_classes = df.loc[df['Предметы ЕГЭ'] == 'Физика', 'Специальность/направление'].unique()
social_science_classes = df.loc[df['Предметы ЕГЭ'] == 'Обществознание', 'Специальность/направление'].unique()

In [5]:
computer_science = []
nature_science = []
social_science = []

for student in data:
#     print(classes[np.where(student['result'] == 1)[0][0]] + ' ' + str(np.where(student['result'] == 1)[0][0]))
    if classes[np.where(student['result'] == 1)[0][0]] in computer_science_classes:
        computer_science.append(student)
    if classes[np.where(student['result'] == 1)[0][0]] in nature_science_classes:
        nature_science.append(student)
    if classes[np.where(student['result'] == 1)[0][0]] in social_science_classes:
        social_science.append(student)

In [6]:
data_computer_scince = np.array([['Направление', 'Информатика', 'Математика', 'Русский язык']])
data_nature_science = np.array([['Направление', 'Физика', 'Математика', 'Русский язык']])
data_social_science = np.array([['Направление', 'Обществознание', 'Математика', 'Русский язык']])


for student in computer_science:
    data_computer_scince = np.append(data_computer_scince, 
                                    [[classes[np.where(student['result'] == 1)[0][0]],
                                     student['features'][1], student['features'][0], student['features'][2]]],
                                    axis = 0)
    
plot_data_computer_scince = pd.DataFrame(
    data = data_computer_scince[1:,0:],
    columns = data_computer_scince[0,0:])
#-----------------------------------------------------------
for student in nature_science:
    data_nature_science = np.append(data_nature_science, 
                                    [[classes[np.where(student['result'] == 1)[0][0]],
                                     student['features'][3], student['features'][0], student['features'][2]]],
                                    axis = 0)
    
plot_data_nature_science = pd.DataFrame(
    data = data_nature_science[1:,0:],
    columns = data_nature_science[0,0:])
#-----------------------------------------------------------
for student in social_science:
    data_social_science = np.append(data_social_science, 
                                    [[classes[np.where(student['result'] == 1)[0][0]],
                                     student['features'][4], student['features'][0], student['features'][2]]],
                                    axis = 0)
    
plot_data_social_science = pd.DataFrame(
    data = data_social_science[1:,0:],
    columns = data_social_science[0,0:])

In [16]:
import plotly.express as px
fig = px.scatter_3d(plot_data_computer_scince, x='Информатика', y='Математика', z='Русский язык',
              color='Направление', width=1920, height=1080)
fig.show()

In [8]:
fig = px.scatter_3d(plot_data_nature_science, x='Физика', y='Математика', z='Русский язык',
              color='Направление')
fig.show()

In [9]:
import plotly.express as px
fig = px.scatter_3d(plot_data_social_science, x='Обществознание', y='Математика', z='Русский язык',
              color='Направление')
fig.show()